In [ ]:
#Check GPU availability

import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU")

GPU is available



*   https://huggingface.co/amazon/chronos-t5-base
*   https://arxiv.org/html/2403.07815v1
*   https://github.com/amazon-science/chronos-forecasting

In [ ]:
!pip install git+https://github.com/amazon-science/chronos-forecasting.git

  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-5kwj0p0e
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-5kwj0p0e
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit d2e0c9d6d57878ebf5af62c1ad3a12b2eb52175a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl 

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving France.csv to France.csv


In [ ]:
import pandas as pd

# Assuming 'yourfile.csv' is the name of your uploaded file
df = pd.read_csv('France.csv') #df.csv, FR.csv. Hungary.csv

# Display the DataFrame
print(df.tail())

      Country ISO3 Code       Datetime (UTC)     Datetime (Local)  \
81067  France       FRA  2024-03-31 19:00:00  2024-03-31 21:00:00   
81068  France       FRA  2024-03-31 20:00:00  2024-03-31 22:00:00   
81069  France       FRA  2024-03-31 21:00:00  2024-03-31 23:00:00   
81070  France       FRA  2024-03-31 22:00:00  2024-04-01 00:00:00   
81071  France       FRA  2024-03-31 23:00:00  2024-04-01 01:00:00   

       Price (EUR/MWhe)  
81067             26.12  
81068             24.53  
81069             32.35  
81070             40.70  
81071             32.10  


In [ ]:
import pandas as pd  # requires: pip install pandas
import torch
from chronos import ChronosPipeline

pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cuda",  # use "cpu" for CPU inference and "mps" for Apple Silicon
    torch_dtype=torch.bfloat16,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/185M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

## Test input=96, pred_length=96 ##

In [ ]:
# context must be either a 1D tensor, a list of 1D tensors,
# or a left-padded 2D tensor with batch as the first dimension
# forecast shape: [num_series, num_samples, prediction_length]
#forecast = pipeline.predict(
#    context=torch.tensor(df["Price (EUR/MWhe)"]),
#    prediction_length=96, #error meessage for 96: we recommend keeping prediction length<=64
    #You can turn off this check by setting `limit_prediction_length=False`.
#    limit_prediction_length=False,
#    num_samples=32
#$)

In [ ]:
print(ChronosPipeline.predict.__doc__)


        Get forecasts for the given time series.

        Parameters
        ----------
        context
            Input series. This is either a 1D tensor, or a list
            of 1D tensors, or a 2D tensor whose first dimension
            is batch. In the latter case, use left-padding with
            ``torch.nan`` to align series of different lengths.
        prediction_length
            Time steps to predict. Defaults to what specified
            in ``self.model.config``.
        num_samples
            Number of sample paths to predict. Defaults to what
            specified in ``self.model.config``.
        temperature
            Temperature to use for generating sample tokens.
            Defaults to what specified in ``self.model.config``.
        top_k
            Top-k parameter to use for generating sample tokens.
            Defaults to what specified in ``self.model.config``.
        top_p
            Top-p parameter to use for generating sample tokens.
            

## Metrics evaluation on the whole dataset ##

In [ ]:
import numpy as np
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle

# Assuming `pipeline` is your Chronos pipeline and `df` is your DataFrame

# Function to calculate RMSE
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Initialize lists to store predictions, actual values, and window metrics
predictions = []
actuals = []
window_mae = []
window_rmse = []
window_mse = []
window_predictions = []  # To store the median predictions for each window
window_actuals = []      # To store the actual values for each window

# Rolling window parameters
input_length = 96
prediction_length = 96
num_samples = 1 #32

# Get the last 3000 rows of the DataFrame
df_last_3000 = df.iloc[-3000:]

# Initialize the iteration counter
nb_iter = 1

# Iterate over the last 3000 rows using a rolling window
for start_idx in range(len(df_last_3000) - input_length - prediction_length + 1):
    # Define the end index for the input and prediction window
    end_idx = start_idx + input_length
    pred_end_idx = end_idx + prediction_length

    # Extract the input sequence
    input_sequence = df_last_3000["Price (EUR/MWhe)"].to_numpy()[start_idx:end_idx]
    # Extract the actual prediction sequence
    actual_sequence = df_last_3000["Price (EUR/MWhe)"].to_numpy()[end_idx:pred_end_idx]

    # Convert input sequence to tensor
    input_tensor = torch.tensor(input_sequence, dtype=torch.float32)

    # Make prediction
    forecast = pipeline.predict(
        context=input_tensor,
        prediction_length=prediction_length,
        limit_prediction_length=False,
        num_samples=num_samples
    )

    # Store the median predictions (each sample over samples)
    median_predictions = np.quantile(forecast[0].numpy(), [0.5], axis=0)
    median_predictions=median_predictions.flatten()
    predictions.append(median_predictions)
    actuals.append(actual_sequence)

    # Store window predictions and actuals
    window_predictions.append(median_predictions)
    window_actuals.append(actual_sequence)

    # Calculate metrics using median predictions for the current window
    window_mae.append(mean_absolute_error(actual_sequence, median_predictions))
    window_rmse.append(root_mean_squared_error(actual_sequence, median_predictions))
    window_mse.append(mean_squared_error(actual_sequence, median_predictions))

    # Print the current iteration number with prefix 'nb'
    print(f"nb{nb_iter}")
    nb_iter=nb_iter+1

# Convert lists to numpy arrays for easier metric calculation
predictions = np.array(predictions)
actuals = np.array(actuals)

# Calculate global performance metrics as the mean of window metrics
global_mae = np.mean(window_mae)
global_rmse = np.mean(window_rmse)
global_mse = np.mean(window_mse)

# Store predictions and metrics in a dictionary
results = {
    "predictions": predictions,
    "actuals": actuals,
    "window_predictions": window_predictions,
    "window_actuals": window_actuals,
    "metrics": {
        "Global MAE": global_mae,
        "Global RMSE": global_rmse,
        "Global MSE": global_mse
    }
}

# Print the global metrics
print("Global MAE:", global_mae)
print("Global RMSE:", global_rmse)
print("Global MSE:", global_mse)

#To save: results

# Save dictionary as a pickle file
with open('results_Chronos_GPU_France.pkl', 'wb') as pickle_file:
    pickle.dump(results, pickle_file)

# Download the file to your local machine
files.download(f'results_Chronos_GPU_France.pkl')

/usr/local/lib/python3.10/dist-packages/chronos/chronos.py:503: UserWarning: We recommend keeping prediction length <= 64. The quality of longer predictions may degrade since the model is not optimized for it. 
  warnings.warn(msg)


nb1


/usr/local/lib/python3.10/dist-packages/chronos/chronos.py:503: UserWarning: We recommend keeping prediction length <= 64. The quality of longer predictions may degrade since the model is not optimized for it. 
  warnings.warn(msg)


nb2


/usr/local/lib/python3.10/dist-packages/chronos/chronos.py:503: UserWarning: We recommend keeping prediction length <= 64. The quality of longer predictions may degrade since the model is not optimized for it. 
  warnings.warn(msg)


...

nb2808


/usr/local/lib/python3.10/dist-packages/chronos/chronos.py:503: UserWarning: We recommend keeping prediction length <= 64. The quality of longer predictions may degrade since the model is not optimized for it. 
  warnings.warn(msg)


nb2809
Global MAE: 22.712902930636254
Global RMSE: 27.591637859946307
Global MSE: 956.6527146730759


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#import pickle

#To save: results

# Save dictionary as a pickle file
#with open('results_Chronos_GPU_France.pkl', 'wb') as pickle_file:
#    pickle.dump(results, pickle_file)

# To load the dictionary back
#with open('results.pkl', 'rb') as pickle_file:
 #   loaded_dict = pickle.load(pickle_file)

#print(loaded_dict)

In [ ]:
predictions = results.get("predictions", "Predictions not found.")

In [ ]:
print("Predictions:")
print(predictions)

Predictions:
[[104.89957428 108.01539612 105.93817902 ... 116.41703033 106.02265167
   98.74658966]
 [100.83055115 103.94901276 102.90952301 ... 104.47042084  98.32510376
   93.20400238]
 [ 97.77826691  95.69788361  93.61749268 ... 155.85647583 155.85647583
  156.87515259]
 ...
 [ 62.56634521  62.56634521  63.68360138 ...  76.63858795  75.00798035
   73.37737274]
 [ 66.49291992  64.8166275   65.37539673 ...  73.74487305  78.4645462
   75.51474762]
 [ 66.49710083  65.37950134  65.37950134 ...   5.75665665   5.31383705
    5.31383705]]
